In [ ]:
import pandas as pd
import spacy 
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy import displacy
from spacy.matcher import PhraseMatcher
import time



In [ ]:
%%time

import re
def extractMMSE(note, matcher):
    score_pos = note.find('/30')
    
    if score_pos >= 0:
        return note[score_pos-2:score_pos]
    else:
        score_pos = note.find(' out of 30')
        if score_pos >= 0:
            return note[score_pos-2:score_pos]
    return None

for typ in ['Prg', 'Vis']:
  prg = open(f"notes_{typ}.txt").read()
  print(len(prg), type(prg))

  header = prg.split('\n')[0]
  print('prg', header)
  prg_notes = prg.split('[report_end]')
  print(len(prg_notes))

  dict_list = []

  program_starts = time.time()
  now = time.time()
  j = 0

  for i in range(len(prg_notes)):
      if i % 50000 == 0:
          print("PRG Note ", i, len(dict_list), time.time()-program_starts, time.time()-now)
          now = time.time()
      #----------------------------  0------1--------2------3--------4------------5---------------------6-------------7-----------8----------9----------
      note = prg_notes[i].replace('EMPI|EPIC_PMRN|MRN_Type|MRN|Report_Number|Report_Date_Time|Report_Description|Report_Status|Report_Type|Report_Text\n',
                                  '')
      if max(note.lower().find('mmse'), 
              note.lower().find('mental state')) >= 0:
          split_note = note.split('|')
          if len(split_note) == 10:
              note_dict = {'Type': f'{typ}',
                            'EMPI': split_note[0].replace('\n', ''),
                            'Date': split_note[5],
                            'Status': split_note[7],
                            'Author': 'NA',
                            'Institution': 'NA', #split_note[9], # type is really institution
                            'Subject': split_note[6],
                            'Note': note
                          }
              dict_list.append(note_dict)

  program_starts = time.time()
  now = time.time()
  mmse_matcher = None


    def without_keys(d, ex): 
        for i in ex: d.pop(i) 
        return d

    result_list = []
    h, m, mo, u =0, 0, 0, 0
    for i in range(len(dict_list)):
        if i % 1000 == 0:
            print(i, 'h', h, 'm', m, 'moca', mo, 'u', u, 
                  time.time()-program_starts, time.time()-now)
            now = time.time()

        note = dict_list[i]

        mmse = max(note['Note'].lower().find('mmse'), 
                   note['Note'].lower().find('mini-mental'),
                   note['Note'].lower().find('mental state exam'))

        if mmse >= 0:
            m+=1
    #         print(note['Note'][mmse-20:mmse+60])
            mmse_score = extractMMSE(note['Note'][mmse-20:mmse+100], mmse_matcher)

            if mmse_score is None:
    #             print(note['Note'][mmse-20:mmse+100])
                note['MMSE'] = 'NA'
            else:
                print('mmse - ', mmse_score)
                try:
                    if int(mmse_score) <= 30 and int(mmse_score) >= 0:
                        note['MMSE'] = mmse_score
                    else:
                        note['MMSE'] = 'NA'
                        #print('invalid', mmse_score)
                except:
                    note['MMSE'] = 'NA'
                    #print('Except invalid, number', mmse_score)
        else:
            note['MMSE'] = 'NA'

        if m % 100 == 0:
            print(m)
        m+=1

    df = pd.DataFrame.from_dict(dict_list)
    df[['EMPI', 'Date', 'MMSE']].to_csv(f'/Users/brett/Desktop/pd/scores/{typ}_mmse.csv', index=False)